In [1]:
# ! pip install torchtext==0.10.1
! pip install torchtext==0.6.0
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.5 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.2 MB/s eta 

In [2]:
from datasets import load_dataset

import torch
from torchtext import data

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
MODEL_CONFIG = "CONTRASTIVE-2"
print("Using MODEL_CONFIG", MODEL_CONFIG)

Using MODEL_CONFIG CONTRASTIVE-2


In [5]:
PROJECT_ROOT = F"/content/gdrive/My Drive/nlp_project_task_1/"

In [6]:
SEED = 42
MAX_VOCAB_SIZE = 25_000

In [7]:
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
faithdial_dataset = load_dataset("McGill-NLP/FaithDial")

Extracting data files:   0%|          | 0/7 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/3539 [00:00<?, ? examples/s]

Generating test_random_split split:   0%|          | 0/1716 [00:00<?, ? examples/s]

Generating test_topic_split split:   0%|          | 0/1823 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/18357 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3417 [00:00<?, ? examples/s]

Generating valid_random_split split:   0%|          | 0/1666 [00:00<?, ? examples/s]

Generating valid_topic_split split:   0%|          | 0/1751 [00:00<?, ? examples/s]

Dataset faith_dial downloaded and prepared to /root/.cache/huggingface/datasets/McGill-NLP___faith_dial/plain_text/1.0.0/70568c8ab3bbc83b603bce58fa593ab27e7f0d0cde51034e1c2073ff3e14189a. Subsequent calls will reuse this data.


  0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
faithdial_dataset.keys()

dict_keys(['test', 'test_random_split', 'test_topic_split', 'train', 'validation', 'valid_random_split', 'valid_topic_split'])

In [11]:
faithdial_dataset["train"][0]

{'dialog_idx': 0,
 'response': 'Yeah, but once the access to the internet was a rare thing. do you remember?',
 'original_response': "No I could not! I couldn't imagine living when internet access was rare and very few people had it!",
 'history': ['Can you imagine the world without internet access?'],
 'knowledge': 'Internet access was once rare, but has grown rapidly.',
 'BEGIN': ['Hallucination'],
 'VRM': ['Disclosure', 'Ack.']}

In [12]:
def critic_preprocess(dataset):
    """
    Data items transformed into (knowledge, response, is_hallucination)
    """
    new_dataset = []
    for d in dataset:
        # original response
        if d["original_response"] != None:
            new_dataset.append({
                "knowledge": d["knowledge"],
                "response": d["original_response"],
                "hallucination": "yes" if "Hallucination" in d["BEGIN"] else "no",
                "history": " ".join(d["history"]),
                "all": " ".join(d["history"]) + " <eos> " + d["knowledge"] + " <eos> " + d["original_response"]
            })

        # new responses always aren't hallucinations
        new_dataset.append({"knowledge": d["knowledge"],
                            "response": d["response"],
                            "hallucination": "no",
                            "history": " ".join(d["history"]),
                            "all": " ".join(d["history"]) + " <eos> " + d["knowledge"] + " <eos> " + d["response"]
        })
    return new_dataset

In [13]:
import json

def dump_as_json(dataset, filename):
    """
    Takes a list of dicts and dumps it as a json file that torchtext can parse.
    """
    with open(filename, "w") as file:
        for d in dataset:
            file.write(json.dumps(d))
            file.write("\n")


In [14]:
KNOWLEDGE = data.Field(tokenize='spacy', tokenizer_language="en_core_web_sm", include_lengths = True)
RESPONSE = data.Field(tokenize='spacy', tokenizer_language="en_core_web_sm", include_lengths = True)
LABEL = data.LabelField(dtype=torch.float)

In [15]:
dump_as_json(critic_preprocess(faithdial_dataset["test"]), PROJECT_ROOT + "data/faithdial_dataset_test.json")
dump_as_json(critic_preprocess(faithdial_dataset["train"]), PROJECT_ROOT + "data/faithdial_dataset_train.json")
dump_as_json(critic_preprocess(faithdial_dataset["validation"]), PROJECT_ROOT + "data/faithdial_dataset_validation.json")

In [16]:
fields = {"knowledge": ("knowledge", KNOWLEDGE), "response": ("response", RESPONSE), "hallucination": ("label", LABEL)}

dataset = data.TabularDataset.splits(path=PROJECT_ROOT + "data",
                                     train="faithdial_dataset_train.json",
                                     validation="faithdial_dataset_validation.json",
                                     test="faithdial_dataset_test.json",
                                     format="json",
                                     fields=fields)


In [17]:
train_data, valid_data, test_data = dataset

In [18]:
for d in train_data.knowledge:
    print(d)
    break

['Internet', 'access', 'was', 'once', 'rare', ',', 'but', 'has', 'grown', 'rapidly', '.']


In [19]:
KNOWLEDGE.build_vocab(train_data.knowledge,
                      train_data.response,
                      max_size=MAX_VOCAB_SIZE,
                      vectors="fasttext.simple.300d",
                      unk_init=torch.Tensor.normal_)
RESPONSE.vocab = KNOWLEDGE.vocab
LABEL.build_vocab(train_data)

.vector_cache/wiki.simple.vec: 293MB [00:01, 203MB/s]                           
100%|██████████| 111051/111051 [00:12<00:00, 8755.98it/s] 


In [20]:
print(f"Unique tokens in KNOWLEDGE vocabulary: {len(KNOWLEDGE.vocab)}")
print(f"Unique tokens in RESPONSE vocabulary: {len(RESPONSE.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in KNOWLEDGE vocabulary: 25002
Unique tokens in RESPONSE vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [21]:
print(KNOWLEDGE.vocab.freqs.most_common(20))
print(RESPONSE.vocab.freqs.most_common(20))
print(LABEL.vocab.freqs.most_common(20))

[(',', 93250), ('.', 66586), ('the', 62545), ('and', 42629), ('of', 40310), ('a', 35230), ('in', 31301), ('is', 29172), ('to', 23093), ('I', 18559), ("''", 18456), ('that', 14549), ('as', 13986), ('or', 13699), ('are', 11488), ('-', 11324), ('The', 10178), ('for', 10087), ('it', 9883), ("'s", 9402)]
[(',', 93250), ('.', 66586), ('the', 62545), ('and', 42629), ('of', 40310), ('a', 35230), ('in', 31301), ('is', 29172), ('to', 23093), ('I', 18559), ("''", 18456), ('that', 14549), ('as', 13986), ('or', 13699), ('are', 11488), ('-', 11324), ('The', 10178), ('for', 10087), ('it', 9883), ("'s", 9402)]
[('no', 20474), ('yes', 13507)]


In [22]:
print(KNOWLEDGE.vocab.itos[:10])
print(RESPONSE.vocab.itos[:10])
print(LABEL.vocab.itos[:10])

['<unk>', '<pad>', ',', '.', 'the', 'and', 'of', 'a', 'in', 'is']
['<unk>', '<pad>', ',', '.', 'the', 'and', 'of', 'a', 'in', 'is']
['no', 'yes']


In [23]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    sort_key=lambda x: x.knowledge,
    device=device)

In [54]:
from torch import nn

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,
                 n_layers, dropout, pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                      embedding_dim=embedding_dim,
                                      padding_idx=pad_idx)

        self.lstm = nn.LSTM(input_size=embedding_dim,
                            hidden_size=hidden_dim,
                            num_layers=n_layers,
                            bidirectional=True)

        self.fc_k = nn.Linear(in_features=2*hidden_dim,
                              out_features=2*hidden_dim)

        self.fc_r = nn.Linear(in_features=2*hidden_dim,
                              out_features=2*hidden_dim)
        
        # 4-layer feed-forward network
        OUT_0 = 2*hidden_dim
        OUT_1 = hidden_dim
        OUT_2 = hidden_dim
        OUT_3 = output_dim
        self.ffn_0 = nn.Linear(in_features=2*2*hidden_dim,
                               out_features=OUT_0)
        self.ffn_1 = nn.Linear(in_features=OUT_0,
                            out_features=OUT_1)
        self.ffn_2 = nn.Linear(in_features=OUT_1,
                            out_features=OUT_2)
        self.ffn_3 = nn.Linear(in_features=OUT_2,
                            out_features=OUT_3)
        self.activation = nn.ReLU()

        self.dropout = nn.Dropout(dropout)

    # def forward(self, k, k_len, r, r_len):
    #     x_k = self.embedding(k)
    #     x_r = self.embedding(r)

    #     output_k, (hidden_k, cell_k) = self.lstm(x_k)
    #     output_r, (hidden_r, cell_r) = self.lstm(x_r)

    #     # obtain concatenated pooled forward and backward hidden outputs
    #     pooled_r = torch.sum(output_r, 0)
    #     pooled_k = torch.sum(output_k, 0)

    #     hidden = torch.cat((pooled_r, pooled_k), -1)
    #     hidden = self.dropout(hidden)

    #     x_1 = pooled_r.unsqueeze(1);
    #     x_2 = pooled_k.unsqueeze(1);

    #     dist = torch.cdist(x_1, x_2, p=2) # ** 2
    #     dist = dist.squeeze(1)

    #     return self.fc(hidden), dist

    def forward(self, k, k_len, r, r_len):
        x_k = self.embedding(k)
        x_r = self.embedding(r)

        output_k, (hidden_k, cell_k) = self.lstm(x_k)
        output_r, (hidden_r, cell_r) = self.lstm(x_r)

        # obtain concatenated pooled forward and backward hidden outputs
        pooled_r = torch.sum(output_r, 0)
        pooled_k = torch.sum(output_k, 0)

        transf_r = self.fc_r(pooled_r)
        transf_k = self.fc_k(pooled_k)

        hidden = torch.cat((transf_r, transf_k), -1)
        hidden = self.dropout(hidden)

        x_1 = transf_k.unsqueeze(1);
        x_2 = transf_r.unsqueeze(1);

        dist = torch.cdist(x_1, x_2, p=1) # ** 2
        dist = dist.squeeze(1)

        y_0 = self.activation(self.ffn_0(hidden))
        y_1 = self.activation(self.ffn_1(y_0))
        y_2 = self.activation(self.ffn_2(y_1))
        y = self.ffn_3(y_2)

        return y, dist

In [55]:
INPUT_DIM = len(RESPONSE.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
DROPOUT = 0.5
PAD_IDX = RESPONSE.vocab.stoi[RESPONSE.pad_token]


model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
             N_LAYERS, DROPOUT, PAD_IDX)

In [56]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,467,833 trainable parameters


In [57]:
print(RESPONSE.vocab.vectors.shape)
print(KNOWLEDGE.vocab.vectors.shape)

torch.Size([25002, 300])
torch.Size([25002, 300])


In [58]:
model.embedding.weight.data.copy_(RESPONSE.vocab.vectors)

tensor([[ 1.9269,  1.4873,  0.9007,  ..., -2.1268, -0.1341, -1.0408],
        [ 0.7694,  2.5574,  0.5716,  ..., -0.9120,  0.3682,  0.7050],
        [ 0.2013,  0.0104,  0.1623,  ..., -0.0931, -0.1408, -0.1326],
        ...,
        [-0.5805, -0.4524, -0.3285,  ..., -0.7460, -0.1563, -1.3202],
        [ 0.0648,  0.5329, -0.2844,  ...,  0.2478, -0.3729,  0.1253],
        [ 0.3754, -0.2251,  0.0137,  ...,  0.0190, -0.4298, -0.2025]])

In [59]:
UNK_IDX = RESPONSE.vocab.stoi[RESPONSE.unk_token]
PAD_IDX = RESPONSE.vocab.stoi[RESPONSE.pad_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2013,  0.0104,  0.1623,  ..., -0.0931, -0.1408, -0.1326],
        ...,
        [-0.5805, -0.4524, -0.3285,  ..., -0.7460, -0.1563, -1.3202],
        [ 0.0648,  0.5329, -0.2844,  ...,  0.2478, -0.3729,  0.1253],
        [ 0.3754, -0.2251,  0.0137,  ...,  0.0190, -0.4298, -0.2025]])


In [60]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [61]:
M = 5
def contrastive_loss(dists, Y):
    # print(Y)

    # dist = (torch.cdist(X_1, X_2, p=2) ** 2)
    m = torch.full(dists.size(), M).to(device)
    ones = torch.ones(Y.size()).to(device)
    zeros = torch.zeros(dists.size()).to(device)
    stacked = torch.stack((zeros, m - dists))
    max = torch.max(stacked, dim=0, keepdim=True).values.squeeze(0)

    loss = (ones - Y) * dists + Y * max

    # max = torch.maximum(zeros, m - dists)


    # print(dists.size())
    # print(Y.size())
    # print(ones.size())
    # print(zeros.size())
    # print(stacked.size())
    # print(max.size())
    # print(loss.size())

    # we sum the loss
    total_loss = torch.sum(loss)

    return total_loss

In [62]:
# criterion = nn.BCEWithLogitsLoss()
# # criterion = contrastive_loss

# model = model.to(device)
# criterion = criterion.to(device)

In [63]:
LAMBDA_1 = 1
LAMBDA_2 = 0
loss_1 = nn.BCEWithLogitsLoss()
loss_1 = loss_1.to(device)
loss_2 = contrastive_loss

def criterion(preds, dists, label):
    return LAMBDA_1 * loss_1(preds, label) + LAMBDA_2 * loss_2(dists, label)

model = model.to(device)

In [64]:
from sklearn.metrics import f1_score


def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


def binary_f1(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    f1 = f1_score(y.cpu(), rounded_preds.cpu(), average="macro")

    return f1


In [65]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()

        r, r_len = batch.response
        k, k_len = batch.knowledge

        # predictions, dists = model(r, r_len, k, k_len)
        predictions, dists = model(k, k_len, r, r_len)
        predictions = predictions.squeeze(1)
        dists = dists.squeeze(1)

        # loss = criterion(dists, batch.label)
        # loss = criterion(predictions, batch.label)
        loss = criterion(predictions, dists, batch.label)
        acc = binary_accuracy(predictions, batch.label)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [66]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0
    epoch_f1 = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:
            r, r_len = batch.response
            k, k_len = batch.knowledge

            # predictions, dists = model(r, r_len, k, k_len)
            predictions, dists = model(k, k_len, r, r_len)
            predictions = predictions.squeeze(1)
            dists = dists.squeeze(1)

            # loss = criterion(dists, batch.label)
            # loss = criterion(predictions, batch.label)
            loss = criterion(predictions, dists, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            f1 = binary_f1(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_f1 += f1.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_f1 / len(iterator)

In [67]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [68]:
N_EPOCHS = 5
path = PROJECT_ROOT + "/" + MODEL_CONFIG + "_model.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, valid_f1 = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% |')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val. F1: {valid_f1:.3f}')

Epoch: 01 | Epoch Time: 0m 29s
	Train Loss: 0.416 | Train Acc: 80.79% |
	 Val. Loss: 0.367 |  Val. Acc: 83.98% | Val. F1: 0.832
Epoch: 02 | Epoch Time: 0m 29s
	Train Loss: 0.288 | Train Acc: 87.95% |
	 Val. Loss: 0.352 |  Val. Acc: 84.14% | Val. F1: 0.835
Epoch: 03 | Epoch Time: 0m 29s
	Train Loss: 0.218 | Train Acc: 91.10% |
	 Val. Loss: 0.393 |  Val. Acc: 83.50% | Val. F1: 0.826
Epoch: 04 | Epoch Time: 0m 29s
	Train Loss: 0.143 | Train Acc: 94.51% |
	 Val. Loss: 0.434 |  Val. Acc: 83.23% | Val. F1: 0.823
Epoch: 05 | Epoch Time: 0m 29s
	Train Loss: 0.091 | Train Acc: 96.71% |
	 Val. Loss: 0.596 |  Val. Acc: 82.90% | Val. F1: 0.818


In [69]:
model.load_state_dict(torch.load(path, map_location=device))

test_loss, test_acc, test_f1 = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test F1: {test_f1:.2f}')

Test Loss: 0.346 | Test Acc: 84.59% | Test F1: 0.84


In [70]:
# import spacy
# nlp = spacy.load('en_core_web_sm')

# def predict_hallucination(model, knowledge, response, history):
#     model.eval()

#     tokenized_r = [tok.text for tok in nlp.tokenizer(response)]
#     indexed_r = [RESPONSE.vocab.stoi[t] for t in tokenized_r]
#     length_r = [len(indexed_r)]
#     tensor_r = torch.LongTensor(indexed_r).to(device)
#     tensor_r = tensor_r.unsqueeze(1)
#     length_tensor_r = torch.LongTensor(length_r)

#     tokenized_k = [tok.text for tok in nlp.tokenizer(knowledge)]
#     indexed_k = [KNOWLEDGE.vocab.stoi[t] for t in tokenized_k]
#     length_k = [len(indexed_k)]
#     tensor_k = torch.LongTensor(indexed_k).to(device)
#     tensor_k = tensor_k.unsqueeze(1)
#     length_tensor_k = torch.LongTensor(length_k)

#     tokenized_h = [tok.text for tok in nlp.tokenizer(history)]
#     indexed_h = [HISTORY.vocab.stoi[t] for t in tokenized_h]
#     length_h = [len(indexed_h)]
#     tensor_h = torch.LongTensor(indexed_h).to(device)
#     tensor_h = tensor_h.unsqueeze(1)
#     length_tensor_h = torch.LongTensor(length_h)

#     prediction = torch.sigmoid(model(tensor_r, length_tensor_r, tensor_k, length_tensor_k, tensor_h, length_tensor_h))

#     return prediction.item()


In [71]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_hallucination(model, knowledge, response, history):
    model.eval()

    tokenized_r = [tok.text for tok in nlp.tokenizer(response)]
    indexed_r = [RESPONSE.vocab.stoi[t] for t in tokenized_r]
    length_r = [len(indexed_r)]
    tensor_r = torch.LongTensor(indexed_r).to(device)
    tensor_r = tensor_r.unsqueeze(1)
    length_tensor_r = torch.LongTensor(length_r)

    tokenized_k = [tok.text for tok in nlp.tokenizer(knowledge)]
    indexed_k = [KNOWLEDGE.vocab.stoi[t] for t in tokenized_k]
    length_k = [len(indexed_k)]
    tensor_k = torch.LongTensor(indexed_k).to(device)
    tensor_k = tensor_k.unsqueeze(1)
    length_tensor_k = torch.LongTensor(length_k)

    prediction, distance = model(tensor_k, length_tensor_k, tensor_r, length_tensor_r)
    prediction = torch.sigmoid(prediction)

    return prediction.item()


In [72]:
predict_hallucination(model, "Bears are animals.", "I love dogs", "Hi.")

0.9297991394996643

In [73]:
predict_hallucination(model, "Dogs are animals.", "Dogs are animals.", "Hi.")

0.38470324873924255

In [74]:
predict_hallucination(model, "Bears are animals.", "Dogs are animals.", "Hi.")

0.38383740186691284

In [75]:
predict_hallucination(model, "Dogs are animals.", "Dogs are animals.", "What do you know about dogs?")

0.38470324873924255

In [76]:
predict_hallucination(model, "Humans walk dogs daily.", "I was walking my dog last week.", "Do you walk your dog?")

0.9964499473571777

In [77]:
predict_hallucination(model, "Humans walk dogs daily", "Dogs need to be walked daily.", "Do you walk your dog?")

0.5468648076057434

In [78]:
test_data[2].response

['Dylan', "'s", 'Candy', 'Bar', 'is', 'a', 'candy', 'supplier']

In [79]:
predict_hallucination(model, "Dylan's Candy Bar is a candy supplier.", "Dylan's Candy Bar is a candy supplier.", "Do you like Dylan's Candy Bar?")

0.1607043743133545

In [80]:
predict_hallucination(model, "Dylan's Candy Bar is a candy supplier.", "Dylan's Candy Bar is my favorite great brand of candy.", "Do you like Dylan's Candy Bar?")

0.9980512857437134

In [81]:
# print(test_data[2].history)

In [82]:
predict_hallucination(model, "Plants are living things.", "Dylan's Candy Bar is a candy supplier.", "What's your favorite plant?")

0.13019080460071564

In [83]:
faithdial_dataset["train"][3]

{'dialog_idx': 0,
 'response': 'Well, I know that it is defined as the ability to connect and use the internet',
 'original_response': 'What is your favorite thing to do with internet access? I like being able to use my computer and smartphone to use my email and browse the world wide web',
 'history': ['Can you imagine the world without internet access?',
  'Yeah, but once the access to the internet was a rare thing. do you remember?',
  'I do. What else can you tell me ?',
  'Well, I know that more people started using it after some restrictions on internet use were lifited in 1995',
  'That is awesome. I wonder why it was restricted? Probably because they only wanted government and big companies to use it at first.',
  'Yeah. And the Internet actually developed from a project founded by the government called ARPANET, it was responsible for suppoting projects at universities and within the government',
  'Very Interesting. What else can you tell me about Internet Access ?'],
 'knowle

In [84]:
for i in range(10):
    data = faithdial_dataset["train"][i]
    print(data)
    print(predict_hallucination(model, data["knowledge"], data["original_response"], " ".join(data["history"])))
    print(predict_hallucination(model, data["knowledge"], data["response"], " ".join(data["history"])))

{'dialog_idx': 0, 'response': 'Yeah, but once the access to the internet was a rare thing. do you remember?', 'original_response': "No I could not! I couldn't imagine living when internet access was rare and very few people had it!", 'history': ['Can you imagine the world without internet access?'], 'knowledge': 'Internet access was once rare, but has grown rapidly.', 'BEGIN': ['Hallucination'], 'VRM': ['Disclosure', 'Ack.']}
0.9975997805595398
0.10400094091892242
{'dialog_idx': 0, 'response': 'Well, I know that more people started using it after some restrictions on internet use were lifited in 1995', 'original_response': 'It used to be restricted, but around 1995, the restricted were lifted and commercial use of it began', 'history': ['Can you imagine the world without internet access?', 'Yeah, but once the access to the internet was a rare thing. do you remember?', 'I do. What else can you tell me ?'], 'knowledge': 'Use by a wider audience only came in 1995 when restrictions on the 

In [85]:

for i in range(10):
    data = faithdial_dataset["test"][i]
    print(data)
    print(predict_hallucination(model, data["knowledge"], data["original_response"], " ".join(data["history"])))
    print(predict_hallucination(model, data["knowledge"], data["response"], " ".join(data["history"])))

{'dialog_idx': 0, 'response': "I don't know how good they are, but Dylan's Candy Bar has a chain of candy shops in various cities.", 'original_response': "Dylan's Candy Bar is a great brand of candy", 'history': ["I love candy, what's a good brand?"], 'knowledge': "Dylan's Candy Bar is a chain of boutique candy shops and candy supplier currently located in New York City; East Hampton, New York; Los Angeles, Chicago and Miami Beach, as well as in wholesale venues around the globe.", 'BEGIN': ['Hallucination'], 'VRM': ['Disclosure', 'Edification', 'Question']}
0.45580729842185974
0.09255142509937286
{'dialog_idx': 0, 'response': "I don't know, really, but they also are a supplier of candy.", 'original_response': "Dylan's Candy Bar is a candy supplier", 'history': ["I love candy, what's a good brand?", "I don't know how good they are, but Dylan's Candy Bar has a chain of candy shops in various cities.", 'Oh, they do? What kind of candy do they sell?'], 'knowledge': "Dylan's Candy Bar is a

In [86]:

for i in range(10):
    data = faithdial_dataset["test"][i]
    print(data)
    print(predict_hallucination(model, "You have no knowledge.", data["original_response"], " ".join(data["history"])))
    print(predict_hallucination(model, "You have no knowledge.", data["response"], " ".join(data["history"])))

{'dialog_idx': 0, 'response': "I don't know how good they are, but Dylan's Candy Bar has a chain of candy shops in various cities.", 'original_response': "Dylan's Candy Bar is a great brand of candy", 'history': ["I love candy, what's a good brand?"], 'knowledge': "Dylan's Candy Bar is a chain of boutique candy shops and candy supplier currently located in New York City; East Hampton, New York; Los Angeles, Chicago and Miami Beach, as well as in wholesale venues around the globe.", 'BEGIN': ['Hallucination'], 'VRM': ['Disclosure', 'Edification', 'Question']}
0.6279041171073914
0.13841064274311066
{'dialog_idx': 0, 'response': "I don't know, really, but they also are a supplier of candy.", 'original_response': "Dylan's Candy Bar is a candy supplier", 'history': ["I love candy, what's a good brand?", "I don't know how good they are, but Dylan's Candy Bar has a chain of candy shops in various cities.", 'Oh, they do? What kind of candy do they sell?'], 'knowledge': "Dylan's Candy Bar is a 